`Using RTX 4000 series which doesn't support faster communication speedups. Ensuring P2P and IB communications are disabled.`

In [3]:
from IPython.display import Image

## 拓扑

https://juejin.cn/post/7151226884351590431

- X： 自身
- SYS：通过 PCIe 的连接以及 NUMA 节点之间的 SMP 互连（例如，QPI/UPI）
- NODE：通过 PCIe 的连接以及 NUMA 节点内 PCIe 主机桥之间的互连
- PHB：通过 PCIe 和 PCIe 主机桥（通常是 CPU）的连接
- PXB：通过多个 PCIe 桥互连（不跨 PCIe 主机桥）
- PIX：通过最多一个 PCIe 桥互连
- NV# ：通过数量为n组的 NVLinks 互连

In [1]:
!nvidia-smi topo -m

	GPU0	GPU1	CPU Affinity	NUMA Affinity	GPU NUMA ID
GPU0	 X 	SYS	0-63	0		N/A
GPU1	SYS	 X 	0-63	0		N/A

Legend:

  X    = Self
  SYS  = Connection traversing PCIe as well as the SMP interconnect between NUMA nodes (e.g., QPI/UPI)
  NODE = Connection traversing PCIe as well as the interconnect between PCIe Host Bridges within a NUMA node
  PHB  = Connection traversing PCIe as well as a PCIe Host Bridge (typically the CPU)
  PXB  = Connection traversing multiple PCIe bridges (without traversing the PCIe Host Bridge)
  PIX  = Connection traversing at most a single PCIe bridge
  NV#  = Connection traversing a bonded set of # NVLinks


### gpu affinity（亲和性）

- https://github.com/NVIDIA/gpu_affinity

## GPU 与硬件

- bus interface：总线接口
    - 4090 的 bus interface：PCIe 4.0 x16（可以向下兼容到 x8，不过会有性能的损失）
        - PCIe 4.0 是代数；
        - x16：则是带宽；
        - 4.0 * x8 == 3.0 * x16
    - x16 vs. x8 插槽
        - 16个数据传输通道，可以提供最大的数据带宽，
        - x8 插槽有8个数据传输通道，它的带宽是x16的一半。

In [6]:
Image(url='https://i0.wp.com/www.deskdecode.com/wp-content/uploads/2022/07/PCI-Express-X16-vs.-X8-vs.-X4-vs.-X1-Slot-on-motherboard.jpg', width=400)

In [1]:
!lspci | grep -i vga

03:00.0 VGA compatible controller: ASPEED Technology, Inc. ASPEED Graphics Family (rev 52)
18:00.0 VGA compatible controller: NVIDIA Corporation Device 2684 (rev a1)
8a:00.0 VGA compatible controller: NVIDIA Corporation Device 2684 (rev a1)


- 如何查是 x16 还是 x8
    - `!sudo lspci -vvv -s 18:00.0 | grep -i lnkcap`